In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [2]:
#initializing wikipedia tool
api_wrapper= WikipediaAPIWrapper(
    top_k_results= 1,
    doc_content_chars_max= 200
)
wikipedia= WikipediaQueryRun(api_wrapper= api_wrapper )

In [3]:
#own custom tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader= WebBaseLoader(["https://docs.smith.langchain.com/"])
docs= loader.load()
splitter= RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap= 200)
documents= splitter.split_documents(docs)
vectordb= FAISS.from_documents(documents, OllamaEmbeddings(model= 'llama2'))
retriever= vectordb.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool= create_retriever_tool(retriever,
                     name= "langsmith_search",
                     description=  "Search for any information about langchain. To know any information about lancgahin, you must use this tool" )

In [5]:
#Arxiv TOol
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper= ArxivAPIWrapper(top_k_results= 1,
                               doc_content_chars_max= 200 )
arxiv= ArxivQueryRun(
    api_wrapper= arxiv_wrapper
)

In [6]:
#combine tools
tools= [wikipedia, arxiv, retriever_tool]


In [7]:
import os
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

from dotenv import load_dotenv
load_dotenv()
os.environ["HUGGINGFACEHUB_API_TOKEN"]=os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm_1= HuggingFaceEndpoint(
    repo_id= "mistralai/Mistral-7B-Instruct-v0.2",
    temperature= 0.6
)
llm= ChatHuggingFace(llm= llm_1)

c:\Users\sthay\OneDrive\Desktop\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
You are an intelligent agent that can use tools to answer questions. You must respond using the following format exactly by using {tools} tools

Thought: <what you are thinking>
Action: <one of [{tool_names}]>
Action Input: <the input to the action>

When you have enough information to answer the original question, respond with:

Final Answer: <your final answer>

Begin!

{agent_scratchpad}
Question: {input}
"""
)


In [9]:
#Agents
from langchain.agents import create_react_agent
agent= create_react_agent(llm,tools, prompt)

In [10]:
#Agent Executor
from langchain.agents import AgentExecutor
agent_executor= AgentExecutor(
    agent= agent,
    tools= tools,
    verbose= True
)

In [11]:
agent_executor.invoke({"input": "Tell me about langsmith"})



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action::  Thought: I need to use the `langsmith_search` tool to get information about Langsmith.

Action: langsmith_search

Action Input: query:'Langsmith'

Final Answer: <The result of the langsmith_search query>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 